In [1]:
# -*- coding: utf-8 -*-

"""
This program makes learning ev-gmm.
"""

'\nThis program makes learning ev-gmm.\n'

In [2]:
# __future__ module make compatible python2 and python3
from __future__ import division, print_function

# basic modules
import os
import os.path
import time

# for warning ignore
import warnings
#warning.filterwarnings('ignore')

# for file system manupulation 
from shutil import rmtree 
import glob
import argparse

# for save object
import pickle

# for make glaph
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (16, 5)
import librosa.display

# for scientific computing
import numpy as np
from numpy.linalg import norm 
from sklearn.decomposition import PCA
#from sklearn.mixture import GMM # GMM class cannot use after sklearn 0.20.0
import sklearn.mixture
from sklearn.mixture.gaussian_mixture import _compute_precision_cholesky
from sklearn.preprocessing import StandardScaler
import scipy.sparse
from scipy.signal import firwin, lfilter

# for display audio controler
from IPython.display import Audio

# for manuplate audio data
import soundfile as sf
import pyworld as pw
import pysptk
from dtw import dtw
from fastdtw import fastdtw

In [3]:
class WORLD(object):
    """
    WORLD based speech analyzer and synthezer.
    
    Ref : https://github.com/k2kobayashi/sprocket/
    """
    def __init__(self, fs=16000, fftl=1024, shiftms=5.0, minf0=40.0, maxf0=500.0):
        """
        Parameters
        ----------
        fs : int 
            Sampling frequency 
        fftl : int
            FFT length
        shiftms : float
            Shift length [ms]
        minf0 : float
            Floor in F0 estimation
        maxf0 : float
            Seli in F0 estimation
        """
        
        self.fs = fs
        self.fftl = fftl
        self.shiftms = shiftms
        self.minf0 = minf0
        self.maxf0 = maxf0
        
    def analyze(self, x):
        """
        Analyze acoustic featueres.
        
        Parameters
        ----------
        x : array, shape(`T`)
            monoral speech signal in time domain
        
        Returns
        ----------
        f0 : array, shape(`T`)
            F0 sequence
        sp : array, shape(`T`, `fftl / 2 + 1`)
            Spectral envelope sequence
        ap : array, shape(`T`, `fftl / 2 + 1`)
            aperiodicity sequence
        """
        
        f0, time_axis = pw.harvest(x, self.fs, f0_floor=self.minf0,
                                   f0_ceil=self.maxf0, frame_period=self.shiftms)
        sp = pw.cheaptrick(x, f0, time_axis, self.fs, fft_size=self.fftl)
        ap = pw.d4c(x, f0, time_axis, self.fs, fft_size=self.fftl)
        
        assert sp.shape == ap.shape
        
        return f0, sp, ap
    
    def analyze_f0(self, x):
        """
        Analyze f0.
        
        Parameters
        ----------
        x : array, shape(`T`)
            monoral speech signal in time domain
        
        Returns
        ----------
        f0 : array, shape(`T`)
            F0 sequence
        """

        f0, time_axis = pw.harvest(x, self.fs, f0_floor=self.minf0,
                                   f0_ceil=self.maxf0, frame_period=self.shiftms)
        
        assert f0.shape == x.shape()
        
        return f0
    
    def synthesis(self, f0, sp, ap):
        """
        Re-synthesizes a speech waveform from acoustic featueres.
        
        Parameters
        ----------
        f0 : array, shape(`T`)
            F0 sequence
        sp : array, shape(`T`, `fftl / 2 + 1`)
            Spectral envelope sequence
        ap : array, shape(`T`, `fftl / 2 + 1`)
            aperiodicity sequence
        """

        return pw.synthesize(f0, sp, ap, self.fs, frame_period=self.shiftms)

In [4]:
class FeatureExtractor(object):
    """
    Analyze acoustic features from a waveform.
    
    This class may have several types of estimeter like WORLD or STRAIGHT.
    Default type is WORLD.
    
    Ref : https://github.com/k2kobayashi/sprocket/
    """
    
    def __init__(self, analyzer='world', fs=16000, fftl=1024, 
                 shiftms=5.0, minf0=50.0, maxf0=500.0):
        """
        Parameters
        ----------
        analyzer : str
            Analyzer
        fs : int 
            Sampling frequency 
        fftl : int
            FFT length
        shiftms : float
            Shift length [ms]
        minf0 : float
            Floor in F0 estimation
        maxf0 : float
            Seli in F0 estimation
        """
        
        self.analyzer = analyzer
        self.fs = fs
        self.fftl = fftl
        self.shiftms = shiftms
        self.minf0 = minf0
        self.maxf0 = maxf0
    
        if self.analyzer == 'world':
            self.analyzer = WORLD(fs=self.fs, fftl=self.fftl, 
                                  minf0=self.minf0, maxf0=self.maxf0, shiftms=self.shiftms)
        else:
            raise('Analyzer Error : not support type, see FeatureExtractor class.')
        
        self._f0 = None
        self._sp = None
        self._ap = None
        
    def analyze(self, x):
        """
        Analyze acoustic featueres.
        
        Parameters
        ----------
        x : array, shape(`T`)
            monoral speech signal in time domain
        
        Returns
        ----------
        f0 : array, shape(`T`)
            F0 sequence
        sp : array, shape(`T`, `fftl / 2 + 1`)
            Spectral envelope sequence
        ap : array, shape(`T`, `fftl / 2 + 1`)
            aperiodicity sequence
        """
        
        self.x = np.array(x, dtype=np.float)
        self._f0, self._sp, self._ap = self.analyzer.analyze(self.x)
        
        # check f0 < 0
        self._f0[self._f0 < 0] = 0
        
        if np.sum(self._f0) == 0.0:
            print("Warning : F0 values are all zero.")
        
        return self._f0, self._sp, self._ap
    
    def analyze_f0(self, x):
        """
        Analyze f0.
        
        Parameters
        ----------
        x : array, shape(`T`)
            monoral speech signal in time domain
        
        Returns
        ----------
        f0 : array, shape(`T`)
            F0 sequence
        """

        self.x = np.array(x, dtype=np.float)
        self._f0 = self.analyzer.analyze_f0(self.x)

        # check f0 < 0
        self._f0[self._f0 < 0] = 0
        
        if np.sum(self._f0) == 0.0:
            print("Warning : F0 values are all zero.")
        
        return self._f0
    
    def mcep(self, dim=24, alpha=0.42):
        """
        Convert mel-cepstrum sequence from spectral envelope.
        
        Parameters
        ----------
        dim : int
            mel-cepstrum dimension
        alpha : float
            parameter of all-path filter
        
        Returns
        ----------
        mcep : array, shape(`T`, `dim + 1`)
            mel-cepstrum sequence
        """        
        
        self._analyzed_check()
        
        return pysptk.sp2mc(self._sp, dim, alpha)
    
    def codeap(self):
        """
        """
        self._analyzed_check()
        
        return pw.code_aperiodicity(self._ap, self.fs)
    
    def npow(self):
        """
        Normalized power sequence from spectral envelope.
        
        Returns
        ----------
        npow : vector, shape(`T`, `1`)
            Normalized power sequence of the given waveform
        """
        
        self._analyzed_check()
        
        npow = np.apply_along_axis(self._spvec2pow, 1, self._sp)
        
        meanpow = np.mean(npow)
        npow = 10.0 * np.log10(npow / meanpow)
        
        return npow
    
    def _spvec2pow(self, specvec):
        """
        """
        fftl2 = len(specvec) - 1
        fftl = fftl2 * 2
        
        power = specvec[0] + specvec[fftl2]
        for k in range(1, fftl2):
            power += 2.0 * specvec[k]
        power /= fftl
        
        return power
        
    def _analyzed_check(self):
        if self._f0 is None and self._sp is None and self._ap is None:
            raise('Call FeatureExtractor.analyze() before this method.')

In [5]:
class Synthesizer(object):
    """
    Synthesize a waveform from acoustic features.
    
    Ref : https://github.com/k2kobayashi/sprocket/
    """
    def __init__(self, fs=16000, fftl=1024, shiftms=5.0):
        """
        Parameters
        ----------
        fs : int 
            Sampling frequency 
        fftl : int
            FFT length
        shiftms : float
            Shift length [ms]
        """
        
        self.fs = fs
        self.fftl = fftl
        self.shiftms = shiftms
    
    def synthesis(self, f0, mcep, ap, rmcep=None, alpha=0.42):
        """
        Re-synthesizes a speech waveform from acoustic featueres.
        
        Parameters
        ----------
        f0 : array, shape(`T`)
            F0 sequence
        mcep : array, shape(`T`, `dim`)
            mel-cepstrum sequence
        ap : array, shape(`T`, `fftl / 2 + 1`)
            aperiodicity sequence
        rmcep : array, shape(`T`, `dim`)
            array of reference mel-cepstrum sequence
        alpha : float
            parameter of all-path filter
            
        Returns
        ----------
        wav : array,
            syntesized waveform
        """
        
        if rmcep is not None:
            # power modification
            mcep = mod_power(mcep, rmcep, alpha=alpha)
        
        sp = pysptk.mc2sp(mcep, alpha, self.fftl)
        wav = pw.synthesize(f0, sp, ap, self.fs, frame_period=self.shiftms)
        
        return wav
        
    def synthesis_diff(self, x, diffmcep, rmcep=None, alpha=0.42):
        """
        Re-synthesizes a speech waveform from acoustic featueres.
        filtering with a differential mel-cepstrum.
        
        Parameters
        ----------
        x : array, shape(`samples`)
            array of waveform sequence
        diffmcep : array, shape(`T`, `dim`)
            array of differential mel-cepstrum sequence
        rmcep : array, shape(`T`, `dim`)
            array of reference mel-cepstrum sequence
        alpha : float
            parameter of all-path filter
            
        Returns
        ----------
        wav : array,
            syntesized waveform
        """      
        
        x = x.astype(np.float64)
        dim = diffmcep.shape[1] - 1 
        shiftl = int(self.fs / 1000 * self.shiftms)
        
        if rmcep is not None:
            # power modification
            diffmcep = mod_power(rmcep + diffmcep, rmcep, alpha=alpha) - rmcep        
        
        # mc2b = transform mel-cepstrum to MLSA digital filter coefficients.
        b = np.apply_along_axis(pysptk.mc2b, 1, diffmcep, alpha)
        
        mlsa_fil = pysptk.synthesis.Synthesizer(pysptk.synthesis.MLSADF(dim, alpha=alpha),
                                                shiftl)
        wav = mlsa_fil.synthesis(x, b)
        
        return wav
    
    def synthesis_sp(self, f0, sp, ap):
        """
        Re-synthesizes a speech waveform from acoustic featueres.
        
        Parameters
        ----------
        f0 : array, shape(`T`)
            F0 sequence
        spc : array, shape(`T`, `dim`)
            mel-cepstrum sequence
        ap : array, shape(`T`, `fftl / 2 + 1`)
            aperiodicity sequence
            
        Returns
        ----------
        wav : array,
            syntesized waveform
        """      
        
        wav = pw.synthesize(f0, sp, ap, self.fs, frame_period=self.shiftms)
        
        return wav
    
def mod_power(cvmcep, rmcep, alpha=0.42, irlen=256):
    """
    power modification based on inpuulse responce

    Parameters
    ----------
    cvmcep : array, shape(`T`, `dim`)
        array of converted mel-cepstrum
    rmcep : arraym shape(`T`, `dim`)
        array of reference mel-cepstrum
    alpha : float
        parameter of all-path filter
    irlen : int
        Length for IIR filter

    Returns
    ----------
    modified_cvmcep : array, shape(`T`, `dim`)
        array of power modified converted mel-cepstrum
    """

    if rmcep.shape != cvmcep.shape:
        raise ValueError(
            "The shape of the converted and reference mel-cepstrum are different : {} / {}.format(cvmcep.shape, rmcep.shape)"
        )

    # mc2e = Compute energy from mel-cepstrum. e-option
    cv_e = pysptk.mc2e(cvmcep, alpha=alpha, irlen=irlen)
    r_e = pysptk.mc2e(rmcep, alpha=alpha, irlen=irlen)

    dpow = np.log(r_e / cv_e) / 2

    modified_cvmcep = np.copy(cvmcep)
    modified_cvmcep[:, 0] += dpow

    return modified_cvmcep

In [6]:
# def util methods
def melcd(array1, array2):
    """
    calculate mel-cepstrum distortion
    
    Parameters
    ----------
    array1, array2 : array, shape(`T`, `dim`) or shape(`dim`)
        Array of original and target.
    
    Returns
    ----------
    mcd : scala, number > 0
        Scala of mel-cepstrum distoriton
    """
    if array1.shape != array2.shape:
        raise ValueError(
            "The shape of both array are different : {} / {}.format(array1.shape,array2.shape)"
        )    
   
    if array1.ndim == 2:
        diff = array1 - array2
        mcd = 10.0 / np.log(10) * np.mean(np.sqrt(2.0 * np.sum(diff ** 2, axis=1)))
    elif array1.ndim == 1:
        diff = array1 - array2
        mcd = 10.0 / np.log(10) * np.sqrt(2.0 * np.sum(diff ** 2))
    else:
        raise ValueError("Dimension mismatch.")
        
    return mcd

def delta(data, win=[-1.0, 1.0, 0]):
    """
    calculate delta component
    
    Parameters
    ----------
    data : array, shape(`T`, `dim`)
        Array of static matrix sequence.
    win : array, shape(`3`)
        The shape of window matrix.
    
    Returns
    ----------
    delta : array, shape(`T`, `dim`)
        Array of delta matrix sequence.
    """
    
    if data.ndim == 1:
        # change vector into 1d-array
        T = len(data)
        dim = data.ndim
        data = data.reshape(T, dim)
    else:
        T, dim = data.shape
    
    win = np.array(win, dtype=np.float64)
    delta = np.zeros((T, dim))
    
    delta[0] = win[0] * data[0] + win[1] * data[1]
    delta[-1] = win[0] * data[-2] + win[1] * data[-1]
    
    for i in range(len(win)):
        delta[1:T - 1] += win[i] * delta[i:T - 2 + i]
    
    return delta

def static_delta(data, win=[-1.0, 1.0, 0]):
    """
    calculate static and delta component
    
    Parameters
    ----------
    data : array, shape(`T`, `dim`)
        Array of static matrix sequence.
    win : array, shape(`3`)
        The shape of window matrix.
    
    Returns
    ----------
    sddata : array, shape(`T`, `dim * 2`)
        Array of static and delta matrix sequence.
    """
    
    sddata = np.c_[data, delta(data, win)]
    
    assert sddata.shape[1] == data.shape[1] * 2
    
    return sddata

def construct_static_and_delta_matrix(T, D, win=[-1.0, 1.0, 0]):
    """
    calculate static and delta transformation matrix
    
    Parameters
    ----------
    T : scala, `T`
        Scala of time length
    D : scala, `D`
        Scala of the number of dimension.
    win : array, shape(`3`)
        The shape of window matrix.
    
    Returns
    ----------
    W : array, shape(`2 * D * T`, `D * T`)
        Array of static and delta transformation matrix.
    """
    
    static = [0, 1, 0]
    delta = win
    assert len(static) == len(delta)
    
    # generate full W
    DT = D * T
    ones = np.ones(DT)
    row = np.arange(2 * DT).reshape(2 * T, D) # generate serial numbers
    static_row = row[::2] # [1,2,3,4,5] => [1,3,5]
    delta_row = row[1::2] # [1,2,3,4,5] => [2,4]
    col = np.arange(DT)
    
    data = np.array([ones * static[0], ones * static[1],
                      ones * static[2], ones * delta[0],
                      ones * delta[1], ones * delta[2]]).flatten()
    row = np.array([[static_row] * 3, [delta_row] * 3]).flatten()
    col = np.array([[col - D, col, col + D] * 2]).flatten()

    # remove component at first and end frame
    valid_idx = np.logical_not(np.logical_or(col < 0, col >= DT))
    
    W = scipy.sparse.csr_matrix(
        (data[valid_idx], (row[valid_idx], col[valid_idx])), shape=(2 * DT, DT))
    W.eliminate_zeros()
    
    return W
    
def extfrm(data, npow, power_threshold=-20):
    """
    Extract frame over the power threshold
    
    Parameters
    ----------
    data : array, shape(`T`, `dim`)
        array of input data
    npow : array, shape(`T`)
        vector of normalized power sequence
    threshold : scala
        scala of power threshold [dB]
        
    Returns
    ----------
    data : array, shape(`T_ext`, `dim`)
        remaining data after extracting frame
        `T_ext` <= `T`
    """
    T = data.shape[0]
    if T != len(npow):
        raise("Length of two vectors is different.")
        
    valid_index = np.where(npow > power_threshold)
    extdata = data[valid_index]
    assert extdata.shape[0] <= T
    
    return extdata

def estimate_twf(orgdata, tardata, distance='melcd', fast=True, otflag=None):
    """
    time warping function estimator
    
    Parameters
    ----------
    orgdata : array, shape(`T_org`, `dim`)
        array of source feature
    tardata : array, shape(`T_tar`, `dim`)
        array of target feature
    distance : str 
        distance function
    fast : bool
        use fastdtw instead of dtw
    otflag : str
        Alignment into the length of specification
        'org' : alignment into original length
        'tar' : alignment into target length   
    
    Returns
    ----------
    twf : array, shape(`2`, `T`)
        time warping function between original and target
    """
    
    if distance == 'melcd':
        def distance_func(x, y): return melcd(x, y)
    else:
        raise ValueError('this distance method is not support.')
    
    if fast:
        _, path = fastdtw(orgdata, tardata, dist=distance_func)
        twf = np.array(path).T
    else:
        _, _, _, twf = dtw(orgdata, tardata, distance_func) 
        
    if otflag is not None:
        twf = modify_twf(twf, otflag=otflag)
    
    return twf

def align_data(org_data, tar_data, twf):
    """
    get aligned joint feature vector
    
    Parameters
    ----------
    org_data : array, shape(`T_org`, `dim_org`)
        Acoustic feature vector of original speaker
    tar_data : array, shape(`T_tar`, `dim_tar`)
        Acoustic feature vector of target speaker
    twf : array, shape(`2`, `T`)
        time warping function between original and target
        
    Returns
    ----------
    jdata : array, shape(`T_new`, `dim_org + dim_tar`)
        Joint feature vector between source and target
    """
    
    jdata = np.c_[org_data[twf[0]], tar_data[twf[1]]]
    return jdata

def modify_twf(twf, otflag=None):
    """
    align specified length
    
    Parameters
    ----------
    twf : array, shape(`2`, `T`)
        time warping function between original and target
    otflag : str
        Alignment into the length of specification
        'org' : alignment into original length
        'tar' : alignment into target length   
    
    Returns
    ----------
    mod_twf : array, shape(`2`, `T_new`)
        time warping function of modified alignment
    """
    
    if otflag == 'org':
        of, indice = np.unique(twf[0], return_index=True)
        mod_twf = np.c_[of, twf[1][indice]].T
    elif otflag == 'tar':
        tf, indice = np.unique(twf[1], return_index=True)
        mod_twf = np.c_[twf[0][indice], tf].T 
    
    return mod_twf

def low_cut_filter(x, fs, cutoff=70):
    """
    low cut filter
    
    Parameters
    ----------
    x : array, shape('samples')
        waveform sequence
    fs : array, int
        Sampling frequency
    cutoff : float
        cutoff frequency of low cut filter
    
    Returns
    ----------
    lct_x : array, shape('samples')
        Low cut filtered waveform sequence
    """
    
    nyquist = fs // 2
    norm_cutoff = cutoff / nyquist
    
    # low cut filter
    fil = firwin(255, norm_cutoff, pass_zero=False)
    lct_x = lfilter(fil, 1, x)
    
    return lct_x

def extsddata(data, npow, power_threshold=-20):
    """
    get power extract static and delta feature vector
    
    Parameters
    ----------
    data : array, shape(`T`, `dim`)
        acoustic feature vector
    npow : array, shape(`T`)
        normalized power vector
    power_threshold : float
        power threshold
    
    Returns
    ----------
    extsddata : array, shape(`T_new`, `dim * 2`)
        silence remove static and delta feature vector
    """
    
    extsddata = extfrm(static_delta(data), npow, power_threshold=power_threshold)
    return extsddata

def transform_jnt(array_list):
    num_files = len(array_list)
    for i in range(num_files):
        if i == 0:
            jnt = array_list[i]
        else:
            jnt = np.r_[jnt, array_list[i]]
    return jnt
    

In [7]:
class F0statistics(object):
    """
    Estimate F0 statistics and convert F0
    """
    def __init__(self):
        pass
    
    def estimate(self, f0list):
        """
        estimate F0 statistics from list of f0
        
        Parameters
        ----------
        f0list : list, shape(`f0num`)
            List of several F0 sequence
        
        Returns
        ----------
        f0stats : array, shape(`[mean, std]`)
            values of mean and standard deviation for log f0
        """
        
        n_files = len(f0list)
        for i in range(n_files):
            f0 = f0list[i]
            nonzero_indices = np.nonzero(f0)
            if i == 0:
                f0s = np.log(f0[nonzero_indices])
            else:
                f0s = np.r_[f0s, np.log(f0[nonzero_indices])]
        
        f0stats = np.array([np.mean(f0s), np.std(f0s)])
        
        return f0stats

    def convert(self, f0, orgf0stats, tarf0stats):
        """
        convert F0 based on F0 statistics
        
        Parameters
        ----------
        f0 : array, shape(`T`, `1`)
            array of F0 sequence
        orgf0stats : array, shape(`[mean, std]`)
            vectors of mean and standard deviation of log f0 for original speaker
        tarf0stats : array, shape(`[mean, std]`)
            vectors of mean and standard deviation of log f0 for target speaker
        
        Returns
        ----------
        cvf0 : array, shape(`T`, `1`)
            array of converted F0 sequence
        """
        
        # get length and dimension
        T = len(f0)
        
        # perform f0 conversion
        cvf0 = np.zeros(T)
        
        nonzero_indices  = f0 > 0
        cvf0[nonzero_indices] = np.exp((tarf0stats[1] / orgf0stats[1]) 
                                       * (np.log(f0[nonzero_indices])) 
                                       - orgf0stats[0] + tarf0stats[0])
        
        return cvf0

In [8]:
class GV(object):
    """
    Estimate statistics and perform postfilter based on the GV statistics.
    """
    def __init__(self):
        pass
    
    def estimate(self, datalist):
        """
        estimate GV statistics from list of data
        
        Parameters
        ----------
        datalist : list, shape(`num_data`)
            List of several data ([T, dim]) sequence
        
        Returns
        ----------
        gvstats : array, shape(`2`, `dim`)
            array of mean and standard deviation for GV
        """
        
        n_files = len(datalist)
        
        var = []
        for i in range(n_files):
            data = datalist[i]
            var.append(np.var(data, axis=0))
            
        # calculate vm and vv
        vm = np.mean(np.array(var), axis=0)
        vv = np.var(np.array(var), axis=0)
        gvstats = np.r_[vm, vv]
        gvstats = gvstats.reshape(2, len(vm))
        
        return gvstats

    def postfilter(self, data, gvstats, cvgvstats=None, alpha=1.0, startdim=1):
        """
        perform postfilter based on GV statistics into data
        
        Parameters
        ----------
        data : array, shape(`T`, `dim`)
            array of data sequence
        gvstats : array, shape(`2`, `dim`)
            array of mean and variance for target GV
        cvgvstats : array, shape(`2`, `dim`)
            array of mean and variance for converted GV
        alpha : float
            morphing coefficient between GV transformed data and data.
            alpha * gvpf(data) + (1 - alpha) * data
        startdim : int
            start dimension to perform GV postfilter
        
        Returns
        ----------
        filtered_data : array, shape(`T`, `data`)
            array of GV postfiltered data sequnece
        """
        
        # get length and dimension
        T, dim = data.shape
        assert gvstats is not None
        assert dim == gvstats.shape[1]
        
        # calculate statics of input data
        datamean = np.mean(data, axis=0)
        
        if cvgvstats is None:
            # use variance of the given data
            datavar = np.var(data, axis=0)
        else:
            # use variance of trained gv stats
            datavar = cvgvstats[0]
        
        # perform GV postfilter
        filterd = np.sqrt(gvstats[0, startdim:] / datavar[startdim:]) * (data[:, startdim:] - datamean[startdim:]) + datamean[startdim:]
        
        filterd_data = np.c_[data[:, :startdim], filterd]
        
        return alpha * filterd_data + (1 - alpha) * data

In [9]:
# 0. config path
__versions = "test-stored0.1.1"
__same_path = "./utterance/" + __versions + "/"
pre_stored_source_list = __same_path + 'pre-source/**/V01/T01/**/*.wav'
pre_stored_list = __same_path + "pre/**/V01/T01/**/*.wav"
output_path = __same_path + "output/"

# 1. estimate features
feat = FeatureExtractor()
synthesizer = Synthesizer()

org_f0list = None
org_splist = None
org_mceplist = None
org_aplist = None
org_npowlist = None
org_codeaplist = None

if os.path.exists(output_path + "_org_f0.pickle") \
    and os.path.exists(output_path + "_org_sp.pickle") \
    and os.path.exists(output_path + "_org_ap.pickle") \
    and os.path.exists(output_path + "_org_mcep.pickle") \
    and os.path.exists(output_path + "_org_npow.pickle") \
    and os.path.exists(output_path + "_org_codeap.pickle"):
        
    with open(output_path + "_org_f0.pickle", 'rb') as f:    
        org_f0list = pickle.load(f)
    with open(output_path + "_org_sp.pickle", 'rb') as f:   
        org_splist = pickle.load(f)
    with open(output_path + "_org_ap.pickle", 'rb') as f:  
        org_aplist = pickle.load(f)
    with open(output_path + "_org_mcep.pickle", 'rb') as f:  
        org_mceplist = pickle.load(f)
    with open(output_path + "_org_npow.pickle", 'rb') as f:  
        org_npowlist = pickle.load(f)
    with open(output_path + "_org_codeap.pickle", 'rb') as f:  
        org_codeaplist = pickle.load(f) 
else:
    org_f0list = []
    org_splist = []
    org_mceplist = []
    org_aplist = []
    org_npowlist = []
    org_codeaplist = []
    for files in sorted(glob.iglob(pre_stored_source_list, recursive=True)):
        wavf = files
        x, fs = sf.read(wavf)
        x = np.array(x, dtype=np.float)
        x = low_cut_filter(x, fs, cutoff=70)
        assert fs == 16000

        print("extract acoustic featuers: " + wavf)

        f0, sp, ap = feat.analyze(x)
        mcep = feat.mcep()
        npow = feat.npow()
        codeap = feat.codeap()
        #name, ext = os.path.splitext(wavf)
        #np.save(name + "_or_f0", f0)
        #np.save(name + "_or_sp", sp)
        #np.save(name + "_or_ap", ap)
        #np.save(name + "_or_mcep", mcep)
        #np.save(name + "_or_codeap", codeap)
        org_f0list.append(f0)
        org_splist.append(sp)
        org_mceplist.append(mcep)
        org_aplist.append(ap)
        org_npowlist.append(npow)
        org_codeaplist.append(codeap)

        #wav = synthesizer.synthesis(f0, mcep, ap)
        #wav = np.clip(wav, -32768, 32767)
        #sf.write(name + "_ansys.wav", wav, fs)

    with open(output_path + "_org_f0.pickle", 'wb') as f:    
        pickle.dump(org_f0list, f)
    with open(output_path + "_org_sp.pickle", 'wb') as f:   
        pickle.dump(org_splist, f)
    with open(output_path + "_org_npow.pickle", 'wb') as f:   
        pickle.dump(org_npowlist, f)
    with open(output_path + "_org_ap.pickle", 'wb') as f:  
        pickle.dump(org_aplist, f)
    with open(output_path + "_org_mcep.pickle", 'wb') as f:  
        pickle.dump(org_mceplist, f)
    with open(output_path + "_org_codeap.pickle", 'wb') as f:  
        pickle.dump(org_codeaplist, f) 

mid_f0list = None
mid_mceplist = None
mid_aplist = None
mid_npowlist = None
mid_splist = None
mid_codeaplist = None        

if os.path.exists(output_path + "_mid_f0.pickle") \
    and os.path.exists(output_path + "_mid_sp_0_.pickle") \
    and os.path.exists(output_path + "_mid_ap_0_.pickle") \
    and os.path.exists(output_path + "_mid_mcep.pickle") \
    and os.path.exists(output_path + "_mid_npow.pickle") \
    and os.path.exists(output_path + "_mid_codeap.pickle"):
        
    with open(output_path + "_mid_f0.pickle", 'rb') as f:    
        mid_f0list = pickle.load(f)
    for i in range(0, len(org_splist)*21, len(org_splist)):  
        with open(output_path + "_mid_sp_{}_.pickle".format(i), 'rb') as f:
            temp_splist = pickle.load(f)
            if mid_splist is None:
                mid_splist = temp_splist
            else:
                mid_splist = mid_splist + temp_splist
    for i in range(0, len(org_aplist)*21, len(org_aplist)):  
        with open(output_path + "_mid_ap_{}_.pickle".format(i), 'rb') as f:
            temp_aplist = pickle.load(f)
            if mid_aplist is None:
                mid_aplist = temp_aplist
            else:
                mid_aplist = mid_aplist + temp_aplist    
    with open(output_path + "_mid_mcep.pickle", 'rb') as f:  
        mid_mceplist = pickle.load(f)
    with open(output_path + "_mid_npow.pickle", 'rb') as f:  
        mid_npowlist = pickle.load(f)
    with open(output_path + "_mid_codeap.pickle", 'rb') as f:  
        mid_codeaplist = pickle.load(f) 
else:        
    mid_f0list = []
    mid_mceplist = []
    mid_aplist = []
    mid_npowlist = []
    mid_splist = []
    mid_codeaplist = []

    for files in sorted(glob.iglob(pre_stored_list, recursive=True)):
        wavf = files
        x, fs = sf.read(wavf)
        x = np.array(x, dtype=np.float)
        x = low_cut_filter(x, fs, cutoff=70)
        assert fs == 16000

        print("extract acoustic featuers: " + wavf)

        f0, sp, ap = feat.analyze(x)
        mcep = feat.mcep()
        npow = feat.npow()
        codeap = feat.codeap()
        name, ext = os.path.splitext(wavf)
        #np.save(name + "_or_f0", f0)
        #np.save(name + "_or_sp", sp)
        #np.save(name + "_or_ap", ap)
        #np.save(name + "_or_mcep", mcep)
        #np.save(name + "_or_codeap", codeap)
        mid_f0list.append(f0)
        mid_splist.append(sp)
        mid_mceplist.append(mcep)
        mid_aplist.append(ap)
        mid_npowlist.append(npow)
        mid_codeaplist.append(codeap)

        #wav = synthesizer.synthesis(f0, mcep, ap)
        #wav = np.clip(wav, -32768, 32767)
        #sf.write(name + "_ansys.wav", wav, fs)
        
    with open(output_path + "_mid_f0.pickle", 'wb') as f:
        print(f)
        pickle.dump(mid_f0list, f)
    with open(output_path + "_mid_npow.pickle", 'wb') as f:
        print(f)
        pickle.dump(mid_npowlist, f)
    for i in range(0, len(mid_splist), len(org_splist)):
        with open(output_path + "_mid_sp_{}_.pickle".format(i), 'wb') as f:   
            print(f)
            pickle.dump(mid_splist[i:i+len(org_splist)], f)
    for i in range(0, len(mid_aplist), len(org_aplist)):
        with open(output_path + "_mid_ap_{}_.pickle".format(i), 'wb') as f:
            print(f)
            pickle.dump(mid_aplist[i:i+len(org_aplist)], f)
    with open(output_path + "_mid_mcep.pickle", 'wb') as f:
        print(f)
        pickle.dump(mid_mceplist, f)
    with open(output_path + "_mid_codeap.pickle", 'wb') as f:
        print(f)
        pickle.dump(mid_codeaplist, f) 

In [10]:
class GMMTrainer(object):
    """
    this class offers the training of GMM with several types of covariance matrix.
    
    Parameters
    ----------
    n_mix : int 
        the number of mixture components of the GMM
    n_iter : int
        the number of iteration for EM algorithm
    covtype : str
        the type of covariance matrix of the GMM
        'full': full-covariance matrix
    
    Attributes
    ---------
    param : 
        sklearn-based model parameters of the GMM
    """
    
    def __init__(self, n_mix=64, n_iter=100, covtype='full'):
        self.n_mix = n_mix
        self.n_iter = n_iter
        self.covtype = covtype
        
        self.param = sklearn.mixture.GaussianMixture(n_components=self.n_mix,
                                                     covariance_type=self.covtype,
                                                     max_iter=self.n_iter)
        
    def train(self, jnt):
        """
        fit GMM parameter from given joint feature vector
        
        Parametes
        ---------
        jnt : array, shape(`T`, `jnt.shape[0]`)
            joint feature vector of original and target feature vector consisting of static and delta components
        """
        
        if self.covtype == 'full':
            self.param.fit(jnt)
            
        return
    
class GMMConvertor(object):
    """
    this class offers the several conversion techniques such as Maximum Likelihood Parameter Generation (MLPG)
    and Minimum Mean Square Error (MMSE).
    
    Parametes
    ---------
    n_mix : int
        the number of mixture components of the GMM
    covtype : str
        the type of covariance matrix of the GMM
        'full': full-covariance matrix
    gmmmode : str
        the type of the GMM for opening
        `None` : Normal Joint Density - GMM (JD-GMM)
    
    Attributes
    ---------
    param : 
        sklearn-based model parameters of the GMM
    w : shape(`n_mix`)
        vector of mixture component weight of the GMM
    jmean : shape(`n_mix`, `jnt.shape[0]`)
        Array of joint mean vector of the GMM
    jcov : shape(`n_mix`, `jnt.shape[0]`, `jnt.shape[0]`)
        array of joint covariance matrix of the GMM
    """
    
    def __init__(self, n_mix=64, covtype='full', gmmmode=None):
        self.n_mix = n_mix
        self.covtype = covtype
        self.gmmmode = gmmmode
        
    def open_from_param(self, param):
        """
        open GMM from GMMTrainer
        
        Parameters
        ----------
        param : GMMTrainer
            GMMTrainer class
        """
        
        self.param = param
        self._deploy_parameters()
        
        return
    
    def convert(self, data, cvtype='mlpg'):
        """
        convert data based on conditional probability density function
        
        Parametes
        ---------
        data : array, shape(`T`, `dim`)
            original data will be converted
        cvtype : str
            type of conversion technique
            `mlpg` : maximum likelihood parameter generation
            
        Returns
        ----------
        odata : array, shape(`T`, `dim`)
            converted data
        """
        
        # estimate parameter sequence
        cseq, wseq, mseq, covseq = self._gmmmap(data)
        
        if cvtype == 'mlpg':
            odata = self._mlpg(mseq, covseq)
        else:
            raise ValueError('please choose conversion mode in `mlpg`.')
        
        return odata
    
    def _gmmmap(self, sddata):
        # paramete for sequencial data
        T, sddim = sddata.shape
        
        # estimate posterior sequence
        wseq = self.pX.predict_proba(sddata)
        
        # estimate mixture sequence
        cseq = np.argmax(wseq, axis=1)
        
        mseq = np.zeros((T, sddim))
        covseq = np.zeros((T, sddim, sddim))
        for t in range(T):
            # read maximum likelihood mixture component in frame t
            m = cseq[t]
            
            # conditional mean vector sequence
            mseq[t] = self.meanY[m] + self.A[m] @ (sddata[t] - self.meanX[m])
            
            # conditional covariance sequence
            covseq[t] = self.cond_cov_inv[m]
        
        return cseq, wseq, mseq, covseq
    
    def _mlpg(self, mseq, covseq):
        # parameter for sequencial data
        T, sddim = mseq.shape
        
        # prepare W
        W = construct_static_and_delta_matrix(T, sddim // 2)
        
        # prepare D
        D = get_diagonal_precision_matrix(T, sddim, covseq)
        
        # calculate W'D
        WD = W.T @ D
        
        # W'DW
        WDW = WD @ W
        
        # W'Dm
        WDM = WD @ mseq.flatten()
        
        # estimate y = (W'DW)^-1 * W'Dm
        odata = scipy.sparse.linalg.spsolve(WDW, WDM, use_umfpack=False).reshape(T, sddim // 2)
        
        return odata
    
    def _deploy_parameters(self):
        # read JD-GMM parameters from self.param
        self.W = self.param.weights_
        self.jmean = self.param.means_
        self.jcov = self.param.covariances_
        
        # devide GMM parameters into source and target parameters
        sddim = self.jmean.shape[1] // 2
        self.meanX = self.jmean[:, 0:sddim]
        self.meanY = self.jmean[:, sddim:]
        self.covXX = self.jcov[:, :sddim, :sddim]
        self.covXY = self.jcov[:, :sddim, sddim:]
        self.covYX = self.jcov[:, sddim:, :sddim]
        self.covYY = self.jcov[:, sddim:, sddim:]
        
        # change model parameter of GMM into that of gmmmode
        if self.gmmmode is None:
            pass
        else:
            raise ValueError('please choose GMM mode in [None]')
            
        # estimate parameters for conversion
        self._set_Ab()
        self._set_pX()
        
        return
    
    def _set_Ab(self):
        # calculate A and b from self.jmean, self.jcov
        sddim = self.jmean.shape[1] // 2
        
        # calculate inverse covariance for covariance XX in each mixture
        self.covXXinv = np.zeros((self.n_mix, sddim, sddim))
        for m in range(self.n_mix):
            self.covXXinv[m] = np.linalg.inv(self.covXX[m])
            
        # calculate A, b, and conditional covariance given X
        self.A = np.zeros((self.n_mix, sddim, sddim))
        self.b = np.zeros((self.n_mix, sddim))
        self.cond_cov_inv = np.zeros((self.n_mix, sddim, sddim))
        for m in range(self.n_mix):
            # calculate A (A = yxcov_m * xxcov_m^-1)
            self.A[m] = self.covYX[m] @ self.covXXinv[m]
            
            # calculate b (b = mean^Y - A * mean^X)
            self.b[m] = self.meanY[m] - self.A[m] @ self.meanX[m]
            
            # calculate conditional covariance (cov^(Y|X)^-1 = (yycov - A * xycov)^-1)
            self.cond_cov_inv[m] = np.linalg.inv(self.covYY[m] - self.A[m] @ self.covXY[m])
            
        return
    
    def _set_pX(self):
        # probability density function of X 
        self.pX = sklearn.mixture.GaussianMixture(n_components=self.n_mix, covariance_type=self.covtype)
        self.pX.weights_ = self.W
        self.pX.means_ = self.meanX
        self.pX.covariances_ = self.covXX
        
        # following function is required to estimate porsterior
        # p(x | \lambda^(X))
        self.pX.precisions_cholesky_ = _compute_precision_cholesky(self.covXX, self.covtype)
        
        return
    
def get_diagonal_precision_matrix(T, D, covseq):
    return scipy.sparse.block_diag(covseq, format='csr')        

In [11]:
def get_alignment(odata, onpow, tdata, tnpow, opow=-20, tpow=-20, sd=0, cvdata=None, given_twf=None, otflag=None, distance='melcd'):
    """
    get alignment between original and target.
    
    Parameters
    ----------
    odata : array, shape(`T`, `dim`)
        acoustic feature vector of original
    onpow : array, shape(`T`)
        Normalized power vector of original
    tdata : array, shape(`T`, `dim`)
        acoustic feature vector of target
    tnpow : array, shape(`T`)
        Normalized power vector of target
    opow : float
        power threshold of original
    tpow : float
        power threshold of target
    sd : int
        start dimension to be used for alignment
    cvdata : array, shape(`T`, `dim`)
        converted original data
    given_twf : array, shape(`T_new`, `dim * 2`)
        Alignment given twf
    otflag : str
        Alignment into the length of specification
        'org' : alignment into original length
        'tar' : alignment into target length
    distance : str
        Distance function to be used
    
    Returns
    ----------
    jdata : array, shape(`T_new`, `dim * 2`)
        joint static and delta feature vector
    twf : array, shape(`T_new`, `dim * 2`)
        Time warping function
    mcd : float
        Mel-cepstrum distortion between arrays
    """
    
    oexdata = extsddata(odata[:, sd:], onpow, power_threshold=opow)
    texdata = extsddata(tdata[:, sd:], tnpow, power_threshold=tpow)
    
    if cvdata is None:
        align_odata = oexdata
    else:
        cvexdata = extsddata(cvdata, onpow, power_threshold=opow)
        align_odata = cvexdata
    
    if given_twf is None: 
        twf = estimate_twf(align_odata, texdata, distance=distance, otflag=otflag)
    else:
        twf = given_twf
    
    jdata = align_data(oexdata, texdata, twf)
    mcd = melcd(align_odata[twf[0]], texdata[twf[1]])
    
    return jdata, twf, mcd

def align_feature_vectors(odata, onpows, tdata, tnpows, opow=-100, tpow=-100, itnum=3, sd=0, given_twfs=None, otflag=None):
    """
    get alignment to create joint feature vector
    
    Parameters
    ----------
    odata : list, (`num_files`)
        List of original feature vectors
    onpow : list, (`num_files`)
        List of original npows
    tdata : list, (`num_files`)
        List of target feature vectors
    tnpow : list, (`num_files`)
        List of target npows
    opow : float
        power threshold of original
    tpow : float
        power threshold of target
    itnum : int
        the number of iteration
    sd : int
        start dimension of feature vector to be used for alignment
    given_twf : array, shape(`T_new`, `dim * 2`)
        use given alignment while 1st iteration
    otflag : str
        Alignment into the length of specification
        'org' : alignment into original length
        'tar' : alignment into target length
    distance : str
        Distance function to be used
    
    Returns
    ----------
    jdata : array, shape(`T_new`, `dim * 2`)
        joint static and delta feature vector
    twf : array, shape(`T_new`, `dim * 2`)
        Time warping function
    mcd : float
        Mel-cepstrum distortion between arrays
    """
    
    it = 1
    num_files = len(odata)
    cvgmm, cvdata = None, None
    for it in range(1, itnum+1):
        print('{}-th joint feature extraction starts.'.format(it))
        
        # alignment
        twfs, jfvs = [], []
        for i in range(num_files):
            if it == 1 and given_twfs is not None:
                gtwf = given_twfs[i]
            else:
                gtwf = None
            
            if it > 1:
                cvdata = cvgmm.convert(static_delta(odata[i][:, sd:]))
            
            jdata, twf, mcd = get_alignment(odata[i], onpows[i], tdata[i], tnpows[i], opow=opow, tpow=tpow,
                                            sd=sd, cvdata=cvdata, given_twf=gtwf, otflag=otflag)
            twfs.append(twf)
            jfvs.append(jdata)
            print('distortion [dB] for {}-th file: {}'.format(i+1, mcd))
            
        jnt_data = transform_jnt(jfvs)
        
        if it != itnum:
            # train GMM, if not final iteration
            datagmm = GMMTrainer()
            datagmm.train(jnt_data)
            cvgmm = GMMConvertor()
            cvgmm.open_from_param(datagmm.param)
        it += 1
    return jfvs, twfs    

In [12]:
# 2. estimate twf and jnt

for i in range(0, len(mid_mceplist), len(org_mceplist)):
    org_mceps = org_mceplist
    org_npows = org_npowlist
    mid_mceps = mid_mceplist[i:i+len(org_mceps)]
    mid_npows = mid_npowlist[i:i+len(org_npows)]
    assert len(org_mceps) == len(mid_mceps)
    assert len(org_npows) == len(mid_npows)
    assert len(org_mceps) == len(org_npows)
    
    # dtw between original and target 0-th and silence
    print("## alignment mcep 0-th and silence ##")
    jmceps, twfs = align_feature_vectors(org_mceps, org_npows, mid_mceps, mid_npows, opow=-100, tpow=-100, sd=1)
    jnt_mcep = transform_jnt(jmceps)
    
    # create joint feature for codeap using given twfs
    print("## alignment codeap into target length using given twfs ##")
    org_codeaps = org_codeaplist
    mid_codeaps = mid_codeaplist[i:i+len(org_codeaplist)]
    jcodeaps = []
    
    for j in range(len(org_codeaps)):
        # extract codeap joint feature vector
        jcodeap, _, _ = get_alignment(org_codeaps[j], org_npows[j], mid_codeaps[j], mid_npows[j], opow=-100, tpow=-100, given_twf=twfs[j])
        jcodeaps.append(jcodeap)
        
    jnt_codeap = transform_jnt(jcodeaps)
    
    # save joint feature vectors
    with open(output_path + "_jnt_mcep_{}_.pickle".format(i), 'wb') as f:   
        print(f)
        pickle.dump(jnt_mcep, f)
    with open(output_path + "_jnt_codeap_{}_.pickle".format(i), 'wb') as f:   
        print(f)
        pickle.dump(jnt_codeap, f)

## alignment mcep 0-th and silence ##
1-th joint feature extraction starts.
distortion [dB] for 1-th file: 8.431337331321112
distortion [dB] for 2-th file: 7.913621220079705
distortion [dB] for 3-th file: 8.192224367413706
distortion [dB] for 4-th file: 8.38159100906585
distortion [dB] for 5-th file: 8.180223195782405
distortion [dB] for 6-th file: 7.833858870877665
distortion [dB] for 7-th file: 7.932246410591963
distortion [dB] for 8-th file: 7.5194784750436465
distortion [dB] for 9-th file: 7.747421311692206
distortion [dB] for 10-th file: 8.132575923589886
distortion [dB] for 11-th file: 7.706651071017323
distortion [dB] for 12-th file: 8.705950707039811
distortion [dB] for 13-th file: 7.695325052962992
distortion [dB] for 14-th file: 7.985882718021858
distortion [dB] for 15-th file: 8.680840890966518
distortion [dB] for 16-th file: 8.294349164352514
distortion [dB] for 17-th file: 8.66179343971681
distortion [dB] for 18-th file: 8.285278339285925
distortion [dB] for 19-th file: 8.

distortion [dB] for 18-th file: 9.574176929136295
distortion [dB] for 19-th file: 9.098472657043402
distortion [dB] for 20-th file: 9.006791438167374
distortion [dB] for 21-th file: 8.996049936972662
distortion [dB] for 22-th file: 9.9592665249075
distortion [dB] for 23-th file: 9.328618424361936
distortion [dB] for 24-th file: 8.57615592986032
distortion [dB] for 25-th file: 9.411417691593629
distortion [dB] for 26-th file: 7.998929105047954
distortion [dB] for 27-th file: 8.016982664842955
distortion [dB] for 28-th file: 8.884151250621322
distortion [dB] for 29-th file: 9.302110399218035
distortion [dB] for 30-th file: 8.58863439030889
distortion [dB] for 31-th file: 9.062432319256539
distortion [dB] for 32-th file: 9.569307522106202
distortion [dB] for 33-th file: 9.03222042966852
distortion [dB] for 34-th file: 10.516751755179397
distortion [dB] for 35-th file: 9.912723490693036
distortion [dB] for 36-th file: 8.451296968516601
distortion [dB] for 37-th file: 9.546276547293287
dist

distortion [dB] for 36-th file: 7.85369505368226
distortion [dB] for 37-th file: 8.164355547457065
distortion [dB] for 38-th file: 8.024354130791751
distortion [dB] for 39-th file: 7.570872948277819
distortion [dB] for 40-th file: 7.790945143522303
distortion [dB] for 41-th file: 8.8184330926124
distortion [dB] for 42-th file: 7.6074513383750935
distortion [dB] for 43-th file: 8.284989278980511
distortion [dB] for 44-th file: 7.430208415560656
distortion [dB] for 45-th file: 7.776389040600887
distortion [dB] for 46-th file: 9.312694538731977
2-th joint feature extraction starts.
distortion [dB] for 1-th file: 8.526979703694716
distortion [dB] for 2-th file: 7.862133700700592
distortion [dB] for 3-th file: 8.165170262550799
distortion [dB] for 4-th file: 8.980109450080251
distortion [dB] for 5-th file: 8.65289182733874
distortion [dB] for 6-th file: 8.162434276182507
distortion [dB] for 7-th file: 8.91300665802685
distortion [dB] for 8-th file: 8.957587921644917
distortion [dB] for 9-th

distortion [dB] for 7-th file: 9.075844891975784
distortion [dB] for 8-th file: 9.620860734554942
distortion [dB] for 9-th file: 8.852050988865399
distortion [dB] for 10-th file: 8.626831381171835
distortion [dB] for 11-th file: 8.381519678369592
distortion [dB] for 12-th file: 8.179321978931565
distortion [dB] for 13-th file: 8.886433728955394
distortion [dB] for 14-th file: 7.988942155267703
distortion [dB] for 15-th file: 8.35713652695177
distortion [dB] for 16-th file: 8.273919669109679
distortion [dB] for 17-th file: 9.04011657685005
distortion [dB] for 18-th file: 8.583709864364982
distortion [dB] for 19-th file: 9.476889525738995
distortion [dB] for 20-th file: 7.777207577714963
distortion [dB] for 21-th file: 9.220095968885076
distortion [dB] for 22-th file: 8.48550972031516
distortion [dB] for 23-th file: 8.794605458219465
distortion [dB] for 24-th file: 8.49628646725869
distortion [dB] for 25-th file: 9.049168501868786
distortion [dB] for 26-th file: 9.107243475593684
distort

distortion [dB] for 25-th file: 10.26203678857664
distortion [dB] for 26-th file: 9.10375555559339
distortion [dB] for 27-th file: 8.385350500190539
distortion [dB] for 28-th file: 8.67055009912757
distortion [dB] for 29-th file: 8.880038334217211
distortion [dB] for 30-th file: 8.510994268470103
distortion [dB] for 31-th file: 9.209511465439608
distortion [dB] for 32-th file: 8.339244198048842
distortion [dB] for 33-th file: 8.258237365560138
distortion [dB] for 34-th file: 9.601456965231243
distortion [dB] for 35-th file: 8.317542027693127
distortion [dB] for 36-th file: 9.697283263880461
distortion [dB] for 37-th file: 9.394942725504766
distortion [dB] for 38-th file: 8.538470614207084
distortion [dB] for 39-th file: 8.033365096606774
distortion [dB] for 40-th file: 8.647715009256373
distortion [dB] for 41-th file: 8.806780680556537
distortion [dB] for 42-th file: 8.99415868526965
distortion [dB] for 43-th file: 8.851328182645867
distortion [dB] for 44-th file: 7.7417586269424215
di

...

distortion [dB] for 30-th file: 7.952335977352624
distortion [dB] for 31-th file: 8.221215366532057
distortion [dB] for 32-th file: 9.56274745138173
distortion [dB] for 33-th file: 7.877018993275962
distortion [dB] for 34-th file: 8.329099542838014
distortion [dB] for 35-th file: 8.287045609832342
distortion [dB] for 36-th file: 8.354927971003484
distortion [dB] for 37-th file: 8.367293681961716
distortion [dB] for 38-th file: 7.507619028100687
distortion [dB] for 39-th file: 8.842349335259268
distortion [dB] for 40-th file: 8.77373964733225
distortion [dB] for 41-th file: 8.609782485875783
distortion [dB] for 42-th file: 8.212857891911833
distortion [dB] for 43-th file: 8.521881169032293
distortion [dB] for 44-th file: 8.932761630787537
distortion [dB] for 45-th file: 8.51399665851117
distortion [dB] for 46-th file: 9.389810296843645
## alignment codeap into target length using given twfs ##
<_io.BufferedWriter name='./utterance/test-stored0.1.1/output/_twfs_736_.pickle'>
<_io.Buffere

<_io.BufferedWriter name='./utterance/test-stored0.1.1/output/_jnt_codeap_782_.pickle'>
## alignment mcep 0-th and silence ##
1-th joint feature extraction starts.
distortion [dB] for 1-th file: 7.388966427987409
distortion [dB] for 2-th file: 7.512691179268715
distortion [dB] for 3-th file: 7.3556663630226735
distortion [dB] for 4-th file: 7.3919204210856515
distortion [dB] for 5-th file: 7.453807540066443
distortion [dB] for 6-th file: 7.205338965234609
distortion [dB] for 7-th file: 7.0185975678027885
distortion [dB] for 8-th file: 6.961807243325705
distortion [dB] for 9-th file: 7.377330983553646
distortion [dB] for 10-th file: 7.069197811875433
distortion [dB] for 11-th file: 7.523751079096847
distortion [dB] for 12-th file: 7.394678840326077
distortion [dB] for 13-th file: 6.745862665973971
distortion [dB] for 14-th file: 7.5644688467324
distortion [dB] for 15-th file: 7.063875860743181
distortion [dB] for 16-th file: 7.448300558487821
distortion [dB] for 17-th file: 7.8034091833

distortion [dB] for 16-th file: 7.853034289472511
distortion [dB] for 17-th file: 7.333462313707315
distortion [dB] for 18-th file: 7.6203383238383
distortion [dB] for 19-th file: 7.847961860493054
distortion [dB] for 20-th file: 7.874406814501537
distortion [dB] for 21-th file: 8.380369340658687
distortion [dB] for 22-th file: 7.766338296703562
distortion [dB] for 23-th file: 8.436621176120038
distortion [dB] for 24-th file: 8.03163936158291
distortion [dB] for 25-th file: 8.606728515266552
distortion [dB] for 26-th file: 8.283509735217256
distortion [dB] for 27-th file: 7.545206779353587
distortion [dB] for 28-th file: 7.759358644239045
distortion [dB] for 29-th file: 8.221279338193787
distortion [dB] for 30-th file: 7.926109381161654
distortion [dB] for 31-th file: 8.062530226221158
distortion [dB] for 32-th file: 7.929302579350625
distortion [dB] for 33-th file: 8.079305952659016
distortion [dB] for 34-th file: 8.005775945074095
distortion [dB] for 35-th file: 7.4509427738834635
di

distortion [dB] for 34-th file: 7.499218234990468
distortion [dB] for 35-th file: 7.2037492480992125
distortion [dB] for 36-th file: 6.881136830453299
distortion [dB] for 37-th file: 7.6219744001695915
distortion [dB] for 38-th file: 8.251929209843262
distortion [dB] for 39-th file: 8.5076615963192
distortion [dB] for 40-th file: 7.576528645604636
distortion [dB] for 41-th file: 7.845063714701527
distortion [dB] for 42-th file: 7.48638680089594
distortion [dB] for 43-th file: 7.71434609956151
distortion [dB] for 44-th file: 7.190563191568462
distortion [dB] for 45-th file: 6.633365620405408
distortion [dB] for 46-th file: 7.449197985868151
2-th joint feature extraction starts.
distortion [dB] for 1-th file: 8.905937052051025
distortion [dB] for 2-th file: 7.94602176589048
distortion [dB] for 3-th file: 9.331928558297085
distortion [dB] for 4-th file: 9.420656686664731
distortion [dB] for 5-th file: 8.955067547443576
distortion [dB] for 6-th file: 7.892386919696853
distortion [dB] for 7

distortion [dB] for 5-th file: 8.114215371941318
distortion [dB] for 6-th file: 7.922419476841926
distortion [dB] for 7-th file: 9.119925299008457
distortion [dB] for 8-th file: 9.012488263687386
distortion [dB] for 9-th file: 8.952201901676721
distortion [dB] for 10-th file: 8.481295287583393
distortion [dB] for 11-th file: 9.129343990868504
distortion [dB] for 12-th file: 8.861317307697805
distortion [dB] for 13-th file: 7.544465701949976
distortion [dB] for 14-th file: 7.478559412895149
distortion [dB] for 15-th file: 8.287726949404119
distortion [dB] for 16-th file: 8.417925951625923
distortion [dB] for 17-th file: 8.051069357394223
distortion [dB] for 18-th file: 8.644478602324856
distortion [dB] for 19-th file: 8.331997833119306
distortion [dB] for 20-th file: 7.8184705832696855
distortion [dB] for 21-th file: 9.302934039325557
distortion [dB] for 22-th file: 8.564723510278299
distortion [dB] for 23-th file: 8.306846597673408
distortion [dB] for 24-th file: 8.965936263399641
dist

In [13]:
# 3. make EV-GMM

initgmm, initgmm_codeap = None, None
jnt, jnt_codeap = None, []
for i in range(0, len(mid_mceplist), len(org_mceplist)):
    with open(output_path + "_jnt_mcep_{}_.pickle".format(i), 'rb') as f:  
        temp_jnt = pickle.load(f)
        if jnt is None:
            jnt = temp_jnt
        else:
            jnt = np.r_[jnt, temp_jnt]
    with open(output_path + "_jnt_codeap_{}_.pickle".format(i), 'rb') as f:  
        temp_jnt_codeap = pickle.load(f)
        jnt_codeap.append(temp_jnt_codeap)
        
jnt_codeap = transform_jnt(jnt_codeap)
assert jnt.shape[0] == jnt_codeap.shape[0]

# train initial gmm
initgmm = GMMTrainer()
initgmm.train(jnt)

initgmm_codeap = GMMTrainer()
initgmm_codeap.train(jnt_codeap)

KeyboardInterrupt: 

In [ ]:
# get initial gmm params
init_W = initgmm.param.weights_
init_jmean = initgmm.param.means_
init_jcov = initgmm.param.covariances_
sddim = init_jmean.shape[1] // 2
init_meanX = init_jmean[:, :sddim]
init_meanY = init_jmean[:, sddim:]
init_covXX = init_jcov[:, :sddim, :sddim]
init_covXY = init_jcov[:, :sddim, sddim:]
init_covYX = init_jcov[:, sddim:, :sddim]
init_covYY = init_jcov[:, sddim:, sddim:]
fitted_source = init_meanX
fitted_target = init_meanY

# get initial codeap gmm params
init_W_codeap = initgmm_codeap.param.weights_
init_jmean_codeap = initgmm_codeap.param.means_
init_jcov_codeap = initgmm_codeap.param.covariances_
sddim_codeap = init_jmean_codeap.shape[1] // 2
init_meanX_codeap = init_jmean_codeap[:, :sddim_codeap]
init_meanY_codeap = init_jmean_codeap[:, sddim_codeap:]
init_covXX_codeap = init_jcov_codeap[:, :sddim_codeap, :sddim_codeap]
init_covXY_codeap = init_jcov_codeap[:, :sddim_codeap, sddim_codeap:]
init_covYX_codeap = init_jcov_codeap[:, sddim_codeap:, :sddim_codeap]
init_covYY_codeap = init_jcov_codeap[:, sddim_codeap:, sddim_codeap:]
fitted_source_codeap = init_meanX_codeap
fitted_target_codeap = init_meanY_codeap

In [ ]:
depengmm, depengmm_codeap = None, None
depenjnt, depenjnt_codeap = None, None
sv, sv_codeap = [], []
for i in range(0, len(mid_mceplist), len(org_mceplist)):
    with open(output_path + "_jnt_mcep_{}_.pickle".format(i), 'rb') as f:  
        depenjnt = pickle.load(f)
        depengmm = GMMTrainer()
        depengmm.param.weights_ = init_W
        depengmm.param.means_ = init_jmean
        depengmm.param.covariances_ = init_jcov
        depengmm.train(depenjnt)
        sv.append(depengmm.param.means_)
    with open(output_path + "_jnt_codeap_{}_.pickle".format(i), 'rb') as f:  
        depenjnt_codeap = pickle.load(f)
        depengmm_codeap = GMMTrainer()
        depengmm_codeap.param.weights_ = init_W_codeap
        depengmm_codeap.param.means_ = init_jmean_codeap
        depengmm_codeap.param.covariances_ = init_jcov_codeap
        depengmm_codeap.train(depenjnt_codeap)
        sv_codeap.append(depengmm_codeap.param.means_)
sv = np.array(sv)
sv_codeap = np.array(sv_codeap)
np.save("_sv", sv)
np.save("_sv_codeap", sv_codeap)

In [ ]:
# PCA
n_mix = 64
S = int(len(mid_mceplist) / len(org_mceplist))
assert S == 22

source_component, source_features = sv[:, :, :sddim].reshape(S, n_mix * sddim).shape
source_stdsc = StandardScaler() # standarize
source_X_std = source_stdsc.fit_transform(sv[:, :, :sddim].reshape(S, n_mix * sddim)) # calculate covariance
# do PCA
source_cov = source_X_std.T @ source_X_std / (source_component - 1)
source_W, source_V_pca = np.linalg.eig(source_cov)
# transform data to principal area
source_X_pca = source_X_std @ source_V_pca

target_component, target_features = sv[:, :, sddim:].reshape(S, n_mix * sddim).shape
target_stdsc = StandardScaler() # standarize
target_X_std = target_stdsc.fit_transform(sv[:, :, sddim:].reshape(S, n_mix * sddim)) # calculate covariance
# do PCA
target_cov = target_X_std.T @ target_X_std / (target_component - 1)
target_W, target_V_pca = np.linalg.eig(target_cov)
# transform data to principal area
target_X_pca = target_X_std @ target_V_pca

eigenvectors = source_X_pca.reshape((n_mix, sddim, S)), target_X_pca.reshape((n_mix, sddim, S))
source_bias = np.mean(sv[:, :, :sddim], axis=0)
target_bias = np.mean(sv[:, :, sddim:], axis=0)
biasvectors = source_bias.reshape((n_mix, sddim)), target_bias.reshape((n_mix, sddim))

In [ ]:
# PCA for codeap
source_component_codeap, source_features_codeap = sv_codeap[:, :, :sddim_codeap].reshape(S, n_mix * sddim_codeap).shape
source_stdsc_codeap = StandardScaler() # standarize
source_X_std_codeap = source_stdsc_codeap.fit_transform(sv_codeap[:, :, :sddim_codeap].reshape(S, n_mix * sddim_codeap)) # calculate covariance
# do PCA
source_cov_codeap = source_X_std_codeap.T @ source_X_std_codeap / (source_component_codeap - 1)
source_W_codeap, source_V_pca_codeap = np.linalg.eig(source_cov_codeap)
# transform data to principal area
source_X_pca_codeap = source_X_std_codeap @ source_V_pca_codeap

target_component_codeap, target_features_codeap = sv_codeap[:, :, sddim_codeap:].reshape(S, n_mix * sddim_codeap).shape
target_stdsc_codeap = StandardScaler() # standarize
target_X_std_codeap = target_stdsc_codeap.fit_transform(sv_codeap[:, :, sddim_codeap:].reshape(S, n_mix * sddim_codeap)) # calculate covariance
# do PCA
target_cov_codeap = target_X_std_codeap.T @ target_X_std_codeap / (target_component_codeap - 1)
target_W_codeap, target_V_pca_codeap = np.linalg.eig(target_cov_codeap)
# transform data to principal area
target_X_pca_codeap = target_X_std_codeap @ target_V_pca_codeap

eigenvectors_codeap = source_X_pca_codeap.reshape((n_mix, sddim_codeap, S)), target_X_pca_codeap.reshape((n_mix, sddim_codeap, S))
source_bias_codeap = np.mean(sv_codeap[:, :, :sddim_codeap], axis=0)
target_bias_codeap = np.mean(sv_codeap[:, :, sddim_codeap:], axis=0)
biasvectors_codeap = source_bias_codeap.reshape((n_mix, sddim_codeap)), target_bias_codeap.reshape((n_mix, sddim_codeap))

In [ ]:
# 4. estimate statistic features.
for_convert_source = __same_path + 'input/EJM10/V01/T01/TIMIT/000/*.wav' 
for_convert_target = __same_path + 'adaptation/EJF01/V01/T01/ATR503/A/*.wav'

print("## estimate static fetures ##")

src_f0list = []
src_splist = []
src_mceplist = []
src_aplist = []
src_npowlist = []
src_codeaplist = []
for files in sorted(glob.iglob(for_convert_source, recursive=True)):
    wavf = files
    x, fs = sf.read(wavf)
    x = np.array(x, dtype=np.float)
    x = low_cut_filter(x, fs, cutoff=70)
    assert fs == 16000

    print("extract acoustic featuers: " + wavf)

    f0, sp, ap = feat.analyze(x)
    mcep = feat.mcep()
    npow = feat.npow()
    codeap = feat.codeap()
    
    src_f0list.append(f0)
    src_splist.append(sp)
    src_mceplist.append(mcep)
    src_aplist.append(ap)
    src_npowlist.append(npow)
    src_codeaplist.append(codeap)


tar_f0list = []
tar_mceplist = []
tar_aplist = []
tar_npowlist = []
tar_splist = []
tar_codeaplist = []

for files in sorted(glob.iglob(for_convert_target, recursive=True)):
    wavf = files
    x, fs = sf.read(wavf)
    x = np.array(x, dtype=np.float)
    x = low_cut_filter(x, fs, cutoff=70)
    assert fs == 16000

    print("extract acoustic featuers: " + wavf)

    f0, sp, ap = feat.analyze(x)
    mcep = feat.mcep()
    npow = feat.npow()
    codeap = feat.codeap()
    name, ext = os.path.splitext(wavf)

    tar_f0list.append(f0)
    tar_splist.append(sp)
    tar_mceplist.append(mcep)
    tar_aplist.append(ap)
    tar_npowlist.append(npow)
    tar_codeaplist.append(codeap)

In [ ]:
f0stats = F0statistics()
srcf0stats = f0stats.estimate(org_f0list)
tarf0stats = f0stats.estimate(tar_f0list)

gv = GV()
srcgvstats = gv.estimate(org_mceplist)
targvstats = gv.estimate(tar_mceplist)

In [ ]:
# 5. fitting target
epoch = 100

fitgmm = sklearn.mixture.GaussianMixture(n_components=sddim, covariance_type='full', max_iter=100)
fitgmm.weights_ = init_W
print(init_W.shape)
fitgmm.means_ = init_meanY
print(init_meanY.shape)
fitgmm.covariances_ = init_covYY
fitgmm.precisions_cholesky_ = _compute_precision_cholesky(init_covXX, 'full')
fitted_target = None

for i in range(len(tar_mceplist)):
    print("adapt: ", i+1, "/", len(tar_mceplist))
    target = tar_mceplist[i]
    target_pow = target[:, 0]
    target = target[:, 1:]
    for x in range(epoch):
        print("epoch = ", x)
        predict = fitgmm.predict_proba(np.atleast_2d(static_delta(target)))
        print(static_delta(target).shape, predict.shape, target.shape, biasvectors[1][0].shape, sddim)
        
        y = np.sum([predict[:, k: k + 1] * (static_delta(target) - biasvectors[1][k]) for k in range(n_mix)], axis = 1)
        gamma = np.sum(predict, axis = 0)

        left = np.sum([gamma[k] * np.dot(eigenvectors[1][k].T,
                                         np.linalg.solve(fitgmm.covariances_, eigenvectors[1])[k])
                       for k in range(n_mix)], axis=0)
        right = np.sum([np.dot(eigenvectors[1][k].T, 
                               np.linalg.solve(fitgmm.covariances_, y)[k]) 
                        for k in range(n_mix)], axis = 0)
        weight = np.linalg.solve(left, right)

        fitted_target = np.dot(eigenvectors[1], weight) + biasvectors[1]
        fitgmm.means_ = fitted_target

In [ ]:
def mcepconvert(source, weights, jmean, meanX, covarXX, covarXY, covarYX, covarYY,
                fitted_source, fitted_target):
    """
    """
    M = 64
    # set_pX (like a GMMConverter) 
    px = sklearn.mixture.GaussianMixture(n_components=M, covariance_type='full')
    px.weights_ = weights
    px.means_ = meanX
    px.covariances_ = covarXX
    px.precisions_cholesky_ = _compute_precision_cholesky(covarXX, 'full')
    
    # set_Ab (like a GMMConverter)
    sddim = jmean.shape[1] // 2
    covXXinv = np.zeros((M, sddim, sddim))
    for m in range(M):
        covXXinv[m] = np.linalg.inv(covarXX[m])
    A = np.zeros((M, sddim, sddim))
    b = np.zeros((M, sddim))
    cond_cov_inv = np.zeros((M, sddim, sddim))
    for m in range(M):
        A[m] = covarYX[m] @ covXXinv[m]
        b[m] = fitted_target[m] - A[m] @ meanX[m]
        cond_cov_inv[m] = np.linalg.inv(covarYY[m] - A[m] @ covarXY[m])
    
    # _gmmmap
    T, sddim = source.shape
    wseq = px.predict_proba(source)
    cseq = np.argmax(wseq, axis=1)
    mseq = np.zeros((T, sddim))
    covseq = np.zeros((T, sddim, sddim))
    for t in range(T):
        m = cseq[t]
        mseq[t] = fitted_target[m] + A[m] @ (source[t] - meanX[m])
        covseq[t] = cond_cov_inv[m]
    
    # _mlpg
    T, sddim = mseq.shape
    W = construct_static_and_delta_matrix(T, sddim // 2)
    D = get_diagonal_precision_matrix(T, sddim, covseq)
    WD = W.T @ D
    WDW = WD @ W
    WDM = WD @ mseq.flatten()
    
    output = scipy.sparse.linalg.spsolve(WDW, WDM, use_umfpack=False).reshape(T, sddim // 2)
    
    return output

In [ ]:
temp_mcep = src_mceplist[0]
temp_mcep_0th = temp_mcep[:, 0]
temp_mcep = temp_mcep[:, 1:]
sta_mcep = static_delta(temp_mcep)
output_mfcc = np.array(mcepconvert(sta_mcep, init_W, init_jmean, init_meanX, init_covXX, 
                                   init_covXY, init_covYX, init_covYY, fitted_source, fitted_target))

In [ ]:
cvmcep = np.c_[temp_mcep_0th, output_mfcc]

In [ ]:
temp_sm = pysptk.mc2sp(cvmcep, 0.42, 1024)
wav = pw.synthesize(src_f0list[0], temp_sm, src_aplist[0], 16000, frame_period=5)
sf.write("temp.wav", wav, 16000)